# Top URL Per Country Per Date 📊🔎

This notebook demonstrates how to:
1. Run a SQL query against AWS Athena to retrieve data.
2. Insert that data into a local PostgreSQL database.
3. Read back the data from PostgreSQL and display it in a Pandas DataFrame.

In [178]:
import boto3
import time
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

True

#### Explanation 🌐🐍

- **boto3**: AWS SDK for Python, used to interact with AWS services like Athena.  
- **psycopg2**: PostgreSQL database adapter for Python.  
- **pandas**: Data analysis library for tabular data manipulation.  
- **dotenv**: Loads environment variables from a local `.env` file.  


In [179]:
athena_client = boto3.client(
    'athena',
    region_name='eu-west-2',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY'),
    aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))

sql_query = """
SELECT *
FROM (
SELECT server_request_country_code, DATE(datetime), event_url, COUNT(*) AS visit_count, 
RANK() OVER(PARTITION BY server_request_country_code ORDER BY COUNT(*) DESC) AS visit_count_rank
FROM vod_clickstream
WHERE DATE(datetime) BETWEEN CAST('2019-01-01' AS timestamp) AND CAST('2019-01-08' AS timestamp) 
GROUP BY server_request_country_code, DATE(datetime), event_url) ranked
WHERE visit_count_rank = 1
ORDER BY server_request_country_code, visit_count_rank;
"""

query_execution = athena_client.start_query_execution(
    QueryString=sql_query,
    QueryExecutionContext={
        "Database": os.getenv('ATHENA_DATABASE_NAME')
    },
    ResultConfiguration={
        "OutputLocation": os.getenv('S3_OUTPUT_LOCATION')
    }
)


#### Explanation ☁️

- **sql_query**: Defines the SQL statement to execute in Athena.  
- **start_query_execution**: Submits the query to Athena.  
- **QueryExecutionContext**: Tells Athena which database to use.  
- **ResultConfiguration**: Tells Athena where to write query results on S3.


In [180]:

query_status = "QUEUED"
query_execution_id = query_execution["QueryExecutionId"]

while query_status in ["QUEUED", "RUNNING"]:
    query_execution = athena_client.get_query_execution(
        QueryExecutionId=query_execution_id
    )
    query_status = query_execution["QueryExecution"]["Status"]["State"]
    if query_status == "FAILED":
        raise Exception("Athena query failed!")
    time.sleep(1)


#### Explanation ⌛

- **get_query_execution**: Checks the current status of the query using its **QueryExecutionId**.  
- We loop until the query status is no longer `"QUEUED"` or `"RUNNING"`.  
- If the query fails, we raise an exception.  
- Otherwise, we sleep for 1 second between checks to avoid spamming the API.


In [181]:
results = athena_client.get_query_results(
    QueryExecutionId=query_execution_id
)["ResultSet"]["Rows"]


conn = psycopg2.connect(database="etl_bites", user="olikelly", password="i_am_a_password", host="localhost", port="5432")
cursor = conn.cursor()

cursor.execute("""
    DROP TABLE IF EXISTS top_url_per_country_per_date;
    CREATE TABLE top_url_per_country_per_date (
        date VARCHAR,
        country_code VARCHAR,
        url VARCHAR,
        visit_count INTEGER,
        rank INTEGER
    );
""")

conn.commit()


#### Explanation 🐘

- **get_query_results**: Fetches the actual data returned by the SQL query.  
- The result is a dictionary that includes `["ResultSet"]["Rows"]`, which is a list where the first row usually contains column headers, and subsequent rows contain actual data.  
- **psycopg2.connect**: Creates a connection to your local PostgreSQL.  
- We **DROP** the table if it exists, then **CREATE** it anew.  
- This ensures we’re starting with a blank table.  

In [182]:
for row in results[1:]:
    if row["Data"][0]['VarCharValue'] == '' or row["Data"][1]['VarCharValue'] == '' or row["Data"][2]['VarCharValue'] == '' or row["Data"][3]['VarCharValue'] == '' or row["Data"][4]['VarCharValue'] == '':
        print(f"Skipping row: {row}")
        continue

    date = row["Data"][0]["VarCharValue"]
    country_code = row["Data"][1]["VarCharValue"]
    url = row["Data"][2]["VarCharValue"]
    visit_count = int(row["Data"][3]["VarCharValue"])
    visit_count_rank = int(row["Data"][4]["VarCharValue"])

    insert_query = """
        INSERT INTO top_url_per_country_per_date (date, url, country_code, visit_count, rank)
        VALUES (%s, %s, %s, %s, %s);
    """

    try:
        cursor.execute(insert_query, (country_code, url, date, visit_count, visit_count_rank))

    except Exception as e:
        print("Error occurred inserting into analytical DB: %s"% e)
        conn.rollback()  # Rollback the transaction if there's an error

conn.commit()
cursor.close()
conn.close()


Skipping row: {'Data': [{'VarCharValue': ''}, {'VarCharValue': '2019-01-06'}, {'VarCharValue': 'https://www.netflix.com/'}, {'VarCharValue': '44'}, {'VarCharValue': '1'}]}


#### Explanation 💾

- We **skip** the first row (the header).
- For each row, we access `["Data"]`, which is a list of dicts like `{"VarCharValue": "some_value"}`.
- We retrieve each field, convert numeric fields to `int`, and insert them into the table.
- If any error occurs, we rollback the transaction for safety.

In [183]:
conn_string = "dbname=etl_bites user=olikelly password=i_am_a_password host=localhost port='5432'"
query = 'SELECT * FROM top_url_per_country_per_date;'

def read_data_from_postgresql(conn_string, query):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:     
            cur.execute(query)
            data = cur.fetchall()
            colnames = cur.description
            return colnames, data

result = read_data_from_postgresql(conn_string, query)
columns = [column[0] for column in result[0]]
data = result[1]

df = pd.DataFrame(data, columns=columns)
display(df)

,date,country_code,url,visit_count,rank
0,2019-01-05,AD,https://www.netflix.com/ad/,3,1
1,2019-01-06,AD,https://www.netflix.com/search?q=black%20mirr,3,1
2,2019-01-05,AE,https://www.netflix.com/browse,598,1
3,2019-01-04,AF,https://www.netflix.com/browse,12,1
4,2019-01-02,AG,https://www.netflix.com/browse,10,1
...,...,...,...,...,...
279,2019-01-02,YT,https://www.netflix.com/browse,22,1
280,2019-01-05,ZA,https://www.netflix.com/browse,1288,1
281,2019-01-08,ZM,https://www.netflix.com/browse,28,1
282,2019-01-04,ZM,https://www.netflix.com/browse,28,1


#### Explanation 🔎

- We run `SELECT *` to ensure our data was inserted successfully.
- `cursor.fetchall()` gives us a list of tuples.
- We build a DataFrame using the column names from `cursor.description`.
